In [2]:
import torch
# import lietorch
import mrob
import numpy as np
import lietorch
from lietorch import LieGroupParameter
# from grad_utils import mult, mult_mrob

In [5]:
xi = np.random.rand(6)*5
x = np.random.rand(3)*10

T = mrob.geometry.SE3(xi)

In [6]:
def finite_difference_grad(xi, function, eps=1e-9, **kwargs):
    T = mrob.geometry.SE3(xi)
    result = []
    for d_xi in np.eye(6)*eps:
        dT = mrob.geometry.SE3(d_xi)
        L1 = function(dT.mul(T), **kwargs)
        L0 = function(T, **kwargs)
        result.append((L1-L0)/eps)
    return np.array(result[3:] + result[:3]).T

### Gradient of $t_k = Tx_k$ wrt to T

In [7]:
def function1(T, x):
    p = T.transform(x)
    return p @ p.T

In [8]:
num1 = finite_difference_grad(xi=xi, function=function1, x = x)

In [9]:
p = T.transform(x)
analyt1 = 2 * p @ np.concatenate((np.eye(3), -mrob.hat3(p)), 1)

In [10]:
# xi_torch = lietorch.SE3.exp(torch.tensor(xi[3:].tolist() + xi[:3].tolist()).unsqueeze(0).double())
xi_torch = lietorch.SE3.Random(5).double('cpu')
T_auto_ = lietorch.SE3(xi_torch.data)
T_auto = lietorch.LieGroupParameter(T_auto_)

x_torch = torch.from_numpy(x)
p_torch = T_auto[0].act(x_torch) 
loss = p_torch @ p_torch.T
loss.backward()

auto1 = T_auto.grad.numpy()

/mnt/sdb1/home/kbotashev/anaconda3/envs/3dgs/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484801627/work/aten/src/ATen/native/TensorShape.cpp:2981.)
  


In [1]:
qtvec = torch.rand(7)
qtvec = lietorch.SE3(qtvec.unsqueeze(0))

NameError: name 'torch' is not defined

In [34]:
a = lietorch.LieGroupParameter(qtvec)

In [35]:
qtvec.data

tensor([[0.7997, 0.9751, 0.3839, 0.4138, 0.1855, 0.1410, 0.3533]])

In [45]:
a[0].data

tensor([0.7997, 0.9751, 0.3839, 0.6991, 0.3134, 0.2383, 0.5969],
       grad_fn=<SelectBackward0>)

In [54]:
c = qtvec.data[0][3:]

In [55]:
c/torch.linalg.norm(c)

tensor([0.6991, 0.3134, 0.2383, 0.5969])

In [11]:
xi_torch = lietorch.SE3.Random(1).double('cpu')
T_auto_ = lietorch.SE3(xi_torch.data)
T_auto1 = lietorch.LieGroupParameter(T_auto_)

In [25]:
xi_torch = lietorch.SE3.Random(1).double('cpu')
T_auto_ = lietorch.SE3(xi_torch.data)
T_auto2 = lietorch.LieGroupParameter(T_auto_)

In [12]:
data = torch.randn(1,7)
print(data)
a = lietorch.SE3(data)
b = lietorch.LieGroupParameter(a)

tensor([[ 1.0050, -0.0159, -0.4845,  1.0533, -0.1322,  0.2781, -0.4799]])


In [33]:
qvec = torch.tensor([[-1.4687, -1.0565,  0.0730,  0.2187, -0.5488, -0.9733,  0.5268]])
lietorch.SE3.InitFromVec(qvec).matrix()

tensor([[[-0.5866,  0.4990, -0.6379, -1.4687],
         [-0.8041, -0.2646,  0.5324, -1.0565],
         [ 0.0969,  0.8252,  0.5565,  0.0730],
         [ 0.0000,  0.0000,  0.0000,  1.0000]]])

In [40]:
qvec = torch.tensor([[-1.4687, -1.0565,  0.0730,  0.2187, -0.5488, -0.9733,  0.5268]])
lietorch.SE3.InitFromVec(qvec).matrix()

tensor([[[-0.5866,  0.4990, -0.6379, -1.4687],
         [-0.8041, -0.2646,  0.5324, -1.0565],
         [ 0.0969,  0.8252,  0.5565,  0.0730],
         [ 0.0000,  0.0000,  0.0000,  1.0000]]])

In [29]:
T_auto2[0].data.detach().requires_grad_()

tensor([ 0.2797, -0.5209, -0.9508,  0.7729, -0.4066,  0.1601,  0.4600],
       dtype=torch.float64, requires_grad=True)

In [17]:
torch.optim.SGD([T_auto1, T_auto2], 1)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [20]:
T_auto2[0].data

tensor([-0.3364,  0.5099,  0.7412, -0.0979,  0.1525, -0.5201,  0.8346],
       dtype=torch.float64, grad_fn=<SelectBackward0>)

In [24]:
T_auto2[0].translation()

tensor([-0.3364,  0.5099,  0.7412,  1.0000], dtype=torch.float64,
       grad_fn=<ViewBackward0>)

In [41]:
T_auto.grad[0] = torch.rand(1,6).double()

In [42]:
T_auto.grad

tensor([[0.7141, 0.3225, 0.6857, 0.8883, 0.5394, 0.5021],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]], dtype=torch.float64)

In [37]:
auto1

array([[0.50734645, 0.70757806, 0.09913951, 0.95475626, 0.33081645,
        0.86062557],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

In [78]:
analyt1

array([-5.60895913e+00,  2.37715881e+01,  2.20276016e+01, -2.13093590e-14,
       -2.20416463e-15, -4.84318910e-15])

In [90]:
np.allclose(num1, analyt1, atol=1e-5), np.allclose(analyt1, auto1, atol=1e-5), np.allclose(num1, auto1, atol=1e-5)

(True, True, True)

### Gradient of $dir = \frac{x_k + W^Td}{||x_k + W^Td||}$

In [115]:
def function2(T, x):
    d = x - T.inv().transform(np.array([0,0,0]))
    d_ = d/np.linalg.norm(d)
    return d_ 

In [126]:
num2 = finite_difference_grad(xi=xi, function=function2, x=x)[:,:3]

In [127]:
d = x - T.inv().transform(np.array([0,0,0]))
d = np.expand_dims(d,1)

In [128]:
analyt2 = (np.eye(3)/np.linalg.norm(d) - (d @ d.T)/np.linalg.norm(d)**3) @ T.R().T

In [129]:
np.allclose(num2, analyt2)

True

In [130]:
num2.shape

(3, 3)

In [131]:
xi_torch = lietorch.SE3.exp(torch.tensor(xi[3:].tolist() + xi[:3].tolist()).unsqueeze(0).double())
T_auto_ = lietorch.SE3(xi_torch.data)
T_auto = lietorch.LieGroupParameter(T_auto_)

theta_torch = torch.tensor([0,0,0,1]).double()
p_torch = x_torch - T_auto[0].inv().act(theta_torch) 
p_torch = p_torch/torch.linalg.norm(p_torch)
loss = p_torch @ p_torch.T
loss.backward()

auto2 = T_auto.grad.numpy()

In [132]:
auto2

array([[ 3.08203066e-18,  6.77515128e-18, -1.17128735e-17,
         6.16297582e-33,  0.00000000e+00,  7.70371978e-34]])

In [133]:
num2

array([[-0.01090918, -0.04068635,  0.04112976],
       [ 0.03905308,  0.00801074,  0.00129925],
       [-0.04411318,  0.00682199, -0.01859477]])

### Gradient of $p_k = h(P,t)$ projection

In [13]:
P = np.array([[1,0,2,0],
              [0,3,4,0],
              [0,0,5,6],
              [0,0,7,0]])

In [14]:
def h(P,t):
    return np.array([P[0,0]/P[3,2] * (t[0]/t[2]) + P[0,2]/P[3,2],
                     P[1,1]/P[3,2] * (t[1]/t[2]) + P[1,2]/P[3,2]])

In [15]:
def function3(T, x, P):
    return h(P,T.transform(x))

In [16]:
num3 = finite_difference_grad(xi, function=function3, x=x, P=P)

In [17]:
t = T.transform(x)
h_ = h(P, t)
analyt3 = np.array([[P[0,0]/P[3,2]/t[2], 0, -P[0,0]/P[3,2]*t[0]/t[2]**2, -P[0,0]/P[3,2]*t[0]*t[1]/t[2]**2, P[0,0]/P[3,2] + P[0,0]/P[3,2]*t[0]**2/t[2]**2, -P[0,0]/P[3,2]*t[1]/t[2]],
          [0, P[1,1]/P[3,2]/t[2], -P[1,1]/P[3,2]*t[1]/t[2]**2, -P[1,1]/P[3,2] - P[1,1]/P[3,2]*t[1]**2/t[2]**2, P[1,1]/P[3,2]*t[0]*t[1]/t[2]**2, P[1,1]/P[3,2]*t[0]/t[2]]])

In [18]:
np.allclose(num3, analyt3)

True

### Gradient of $T_k = J_k W$ matrix multiplication

In [383]:
def function4(T, Jk, a, b):
    A = np.array([[1,0,0,0],
                  [0,1,0,0],
                  [0,0,1,0]])
    T_k = Jk[:3,:3] @ T.R()
    result = a @ T_k
    # print(result)
    result = result @ b
    # print(result)
    return result

In [384]:
Jk = np.array([[1,0,2],
               [0,3,4],
               [0,0,0]])
Jk_hat = np.eye(4)
a = np.random.randn(3)*100
b = np.random.randn(3)*100
Jk_hat[:3,:3] = Jk
num4 = finite_difference_grad(xi, function=function4, Jk=Jk, a = a, b = b)
print(num4)

[     0.              0.              0.          10060.54299069
  23842.17805229 -67391.73477399]


In [386]:
xi_torch = lietorch.SE3.exp(torch.tensor(xi[3:].tolist() + xi[:3].tolist()).unsqueeze(0).double())
# xi_torch = lietorch.SO3.exp(torch.tensor(xi[:3].tolist()).unsqueeze(0).double())
T_auto_ = lietorch.SE3(xi_torch.data)
T_auto = lietorch.LieGroupParameter(T_auto_)

loss = torch.from_numpy(a) @ (torch.from_numpy(Jk).double() @ T_auto[0].matrix()[:3,:-1])
loss_ = loss @ torch.from_numpy(b)
loss_.backward()

auto4 = T_auto.grad.numpy()

In [387]:
T_auto.grad

tensor([[     0.0000,      0.0000,      0.0000,  10060.5421,  23842.1850,
         -67391.7344]], dtype=torch.float64)

In [388]:
def dExpxi_dxi(T):
    
    wc1 = T.T()[:-1,0]
    wc2 = T.T()[:-1,1]
    wc3 = T.T()[:-1,2]
    d = T.T()[:-1,3]
    result = np.zeros((16,6))
    result[9:12, :3] = np.eye(3)
    
    # result[:3, 3:] = -mrob.hat3(wc1)
    result[:3, 3:] = -mrob.hat3(wc1)
    result[3:6, 3:] = -mrob.hat3(wc2)
    result[6:9, 3:] = -mrob.hat3(wc3)
    result[9:12, 3:] = -mrob.hat3(d)
    # result[9:12, 3:] = -mrob.hat3(d)

    return result

In [389]:
def dExpomega_domega():
    
    wc1 = np.array([1,0,0])
    wc2 = np.array([0,1,0])
    wc3 = np.array([0,0,1])
    
    result = np.zeros((9,3))
    
    result[:3, ] = -mrob.hat3(wc1)
    # result[:3, 3:] = -mrob.hat3(wc1)
    result[3:6, ] = -mrob.hat3(wc2)
    result[6:9, ] = -mrob.hat3(wc3)
    # result[9:12, 3:] = -mrob.hat3(d)
    # result[9:12, 3:] = -mrob.hat3(d)

    return result

In [390]:
analyt42 = np.concatenate((np.zeros((9,3)), np.kron(np.eye(3), Jk) @ dExpxi_dxi(T)[:9,3:]), 1)

In [397]:
dL = np.kron(b.T, a)
W = T.R()

In [428]:
analyt421 = -np.array([0,
          0,
          0,
          - dL[0]*Jk[0,2]*W[1,0] + dL[1]*(Jk[1,1]*W[2,0]-Jk[1,2]*W[1,0]) - dL[3]*Jk[0,2]*W[1,1] + dL[4]*(Jk[1,1]*W[2,1]-Jk[1,2]*W[1,1]) - dL[6]*Jk[0,2]*W[1,2] + dL[7]*(Jk[1,1]*W[2,2]-Jk[1,2]*W[1,2]),
          dL[0]*(-Jk[0,0]*W[2,0] + Jk[0,2]*W[0,0]) + dL[1]*Jk[1,2]*W[0,0] + dL[3]*(-Jk[0,0]*W[2,1] + Jk[0,2]*W[0,1]) + dL[4]*Jk[1,2]*W[0,1] + dL[6]*(-Jk[0,0]*W[2,2] + Jk[0,2]*W[0,2]) + dL[7]*Jk[1,2]*W[0,2],
          dL[0]*Jk[0,0]*W[1,0] - dL[1]*Jk[1,1]*W[0,0] + dL[3]*Jk[0,0]*W[1,1] - dL[4]*Jk[1,1]*W[0,1] + dL[6]*Jk[0,0]*W[1,2] - dL[7]*Jk[1,1]*W[0,2]])

In [431]:
np.allclose(analyt421, num4)

True

In [214]:
A = np.array([[1,0,0,0],
            [0,1,0,0],
            [0,0,1,0]])
analyt4 = np.kron(A, A@Jk_hat) @ dExpxi_dxi(T)

In [215]:
analyt41 = np.kron(A, Jk@A) @ dExpxi_dxi(T)

In [377]:
np.allclose(num4, analyt42)

False

In [379]:
print(analyt42)

[[ 0.          0.          0.         -0.97640219  1.28816907  0.4882011 ]
 [ 0.          0.          0.         -4.48480878  0.88833522 -0.66625141]
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          1.35455151 -0.50013898 -0.67727575]
 [ 0.          0.          0.          1.12845723 -2.05404181  1.54053136]
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          1.10083104  1.75797652 -0.55041552]
 [ 0.          0.          0.          1.9008089   3.31538426 -2.48653819]
 [ 0.          0.          0.          0.          0.          0.        ]]


In [203]:
analyt42.shape

(9, 6)